# Insert AIS data to MongoDB

In [2]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import seaborn as sns
import geopandas as geo
import pprint

In [38]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.AISdata
nari_dynamic = db.nari_dynamic

In [19]:
data = pd.read_csv("../data/[P1] AIS data/nari_dynamic.csv")
data.head()

,sourcemmsi,navigationalstatus,rateofturn,speedoverground,courseoverground,trueheading,lon,lat,t
0,245257000,0.0,0.0,0.1,13.1,36,-4.465718,48.382490,1443650402
1,227705102,15.0,-127.0,0.0,262.7,511,-4.496571,48.382420,1443650403
2,228131600,15.0,-127.0,8.5,263.7,511,-4.644325,48.092247,1443650404
3,228051000,0.0,-127.0,0.0,295.0,511,-4.485108,48.381320,1443650405
4,227574020,15.0,-127.0,0.1,248.6,511,-4.495441,48.383660,1443650406


In [20]:
data['t']=pd.to_datetime(data.t,unit='s')

In [29]:

def add_to_db():
    
    for index, row in data.iterrows():
        point=[row.lon,row.lat]
        doc = {'sourcemmsi':row.sourcemmsi,
               'navigationalstatus':row.navigationalstatus,
               'rateofturn':row.rateofturn,
               'speedoverground':row.speedoverground,
               'courseoverground':row.courseoverground,
               'trueheading':row.trueheading,
               't':row.t,
               'location':{'type':'Point',
                           'coordinates':point}}
        nari_dynamic.insert_one(doc)

    return 'Done'
        


In [30]:
add_to_db()

'Done'

In [41]:
pprint.pprint(nari_dynamic.find_one({'navigationalstatus':7.0}))

{'_id': ObjectId('5ddc74066d592ceb82cc2240'),
 'courseoverground': 119.1,
 'location': {'coordinates': [-4.3249383, 48.098133000000004], 'type': 'Point'},
 'navigationalstatus': 7.0,
 'rateofturn': 0.0,
 'sourcemmsi': 227392000,
 'speedoverground': 0.0,
 't': datetime.datetime(2016, 1, 6, 22, 19, 31),
 'trueheading': 274}


In [34]:
#data2 = pd.read_csv("../data/[P1] AIS data/nari_static.csv")

In [28]:
#data2.head()

,sourcemmsi,imonumber,callsign,shipname,shiptype,tobow,tostern,tostarboard,toport,eta,draught,destination,mothershipmmsi,t
0,304091000,9509255.0,V2GU5,HC JETTE-MARIT,70.0,130.0,30.0,18.0,6.0,04-09 20:00,10.1,BREST,NaN,1443650423
1,228037600,0.0,FIHX,AEROUANT BREIZH,30.0,6.0,9.0,5.0,2.0,00-00 24:60,0.0,,NaN,1443650457
2,228064900,8304816.0,FITO,VN SAPEUR,51.0,21.0,54.0,10.0,6.0,29-09 12:00,5.9,RADE DE BREST,NaN,1443650471
3,227705102,262144.0,FGD5860,BINDY,60.0,9.0,26.0,5.0,4.0,00-00 24:60,0.0,,NaN,1443650474
4,227415000,0.0,FHAF,F/V JEREMI SIMON,90.0,11.0,9.0,3.0,3.0,00-00 24:60,0.0,,NaN,1443650479


In [6]:
nari_static = db.nari_static
#nari_static.insert_many(data2.to_dict('records'))

In [35]:
ships_freq=nari_dynamic.aggregate([
    
        {"$group": {"_id":"$sourcemmsi",
         "count":{"$sum":1}}},
        {"$sort":{"count":-1}}
]
)

# Insert forbidden fishing areas to mongodb

In [31]:
forbidden_fishing_areas=geo.read_file("../data/[C5] Fishing Constraints/fishing_interdiction.shp")

#Camaret (Polygon)
x0,y0=forbidden_fishing_areas.geometry[0].exterior.coords.xy
polygon0=list(list(t) for t in zip(x0,y0))

#Concarneau (MultiPolygon)
polygons1=[]
multipolygon=forbidden_fishing_areas.geometry[1]
for i in range(len(multipolygon)):
    x1,y1=multipolygon[i].exterior.coords.xy
    polygon1=list(list(t) for t in zip(x1,y1))
    polygons1.append(polygon1)


Polygon  0 [[-4.629735984460192, 48.280572114555405], [-4.630281337707765, 48.280765137575024], [-4.6297998340389634, 48.280704972539965], [-4.629735984460192, 48.280572114555405]]
****************************************************************************************************
Polygon  1 [[-4.630434166549479, 48.28081923000895], [-4.764547356637077, 48.328287422482], [-4.763138837785976, 48.328257809609696], [-4.761791903848968, 48.32794209925334], [-4.761164789949124, 48.3280745114966], [-4.7610226764711765, 48.328279370537004], [-4.760578190068594, 48.32823069419233], [-4.760569027252696, 48.328529560243034], [-4.760430327372829, 48.32858480607379], [-4.7599784747849165, 48.32853147145116], [-4.759463560464059, 48.32873137147966], [-4.757626954865676, 48.32854950847424], [-4.757432020449023, 48.32868036628841], [-4.757017447945402, 48.32867379652939], [-4.75685458338439, 48.32883977227292], [-4.756273463227093, 48.32864769666018], [-4.755933810218168, 48.328816479559926], [-4.755

In [32]:
#Ιnsert forbidden areas to mongodb :

forbidden_fishing_areas = db.forbidden_fishing_areas

#Camaret
forbidden_area_0 = {'id':'Camaret',
                    'type':'Polygon',
                    'coordinates':[polygon0]}
forbidden_fishing_areas.insert_one(forbidden_area_0)

#Concarneau
i=0
for poly in polygons1:
    
    forbidden_area_1 = {'id':'Concarneau'+str(i),
                        'type':'Polygon',
                        'coordinates':[poly]}
    forbidden_fishing_areas.insert_one(forbidden_area_1)
    i+=1
    